In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from keras import layers, optimizers
from tensorflow.keras import Model
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.applications.resnet import ResNet50 #loading the resnet50 model
import cv2
import os

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [57]:
#prevents overfitting and handles the imbalance in dataset

# normalizing the images
data_aug = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)

# creating train data
train = data_aug.flow_from_directory('/content/gdrive/MyDrive/PythonProjectWebApp/chest_xray/train', target_size=(150, 150), class_mode='categorical', batch_size=32, shuffle=True )

# creating test data
test = data_aug.flow_from_directory('/content/gdrive/MyDrive/PythonProjectWebApp/chest_xray/test', target_size=(150, 150), class_mode='categorical', batch_size=32, shuffle=True)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [59]:
#building the model

resNet  = ResNet50( input_shape=(150,150,3), include_top=False)

# freezing restNet layers
for layer in resNet.layers:
  layer.trainable = False

l = resNet.output
# adding Maxpool layer
l = GlobalAveragePooling2D()(l)
# adding dense layers
l = Dense(64, activation='relu')(l)
l = Dense(32, activation='relu')(l)

output_layer =layers.Dense(2, activation='sigmoid')(l)

# creating early stopping.
es = EarlyStopping(monitor='val_loss', mode='min', patience = 5, restore_best_weights=True)
# creating model checkpoint
mc = ModelCheckpoint("ResNetModel.h5", monitor='val_loss', mode='min', verbose=1, save_best_only=True)
# creating class weights
cw = {0: 1.0, 1: 1.0}

model = Model(inputs=resNet.input, outputs=output_layer)

model.summary()

plot_model(model, to_file = 'model_plot.png', show_shapes = True, show_layer_names = True)

ValueError: ignored

In [44]:
# compile model
model.compile(optimizer= tf.keras.optimizers.SGD(lr=1e-4, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [45]:
# fitting the model
history = model.fit(train, epochs=12, validation_data=test, callbacks = [es,mc],verbose=1,class_weight=cw, steps_per_epoch= len(train),validation_steps= len(test))

Epoch 1/12
163/163 [==============================] - ETA: 0s - loss: 0.3468 - accuracy: 0.8441
Epoch 1: val_loss improved from inf to 0.53554, saving model to ResNetModel.h5
163/163 [==============================] - 75s 434ms/step - loss: 0.3468 - accuracy: 0.8441 - val_loss: 0.5355 - val_accuracy: 0.7388
Epoch 2/12
163/163 [==============================] - ETA: 0s - loss: 0.1945 - accuracy: 0.9256
Epoch 2: val_loss improved from 0.53554 to 0.53109, saving model to ResNetModel.h5
163/163 [==============================] - 70s 427ms/step - loss: 0.1945 - accuracy: 0.9256 - val_loss: 0.5311 - val_accuracy: 0.7564
Epoch 3/12
163/163 [==============================] - ETA: 0s - loss: 0.1606 - accuracy: 0.9410
Epoch 3: val_loss did not improve from 0.53109
163/163 [==============================] - 69s 423ms/step - loss: 0.1606 - accuracy: 0.9410 - val_loss: 0.5924 - val_accuracy: 0.7532
Epoch 4/12
163/163 [==============================] - ETA: 0s - loss: 0.1424 - accuracy: 0.9457
Epoch

In [46]:
model.save('PNmodelResNet50.h5')

In [104]:
#read the image
# function to load the dataset and return the dataset list
# containing elements of format [scans, target]
targets = ['PNEUMONIA', 'NORMAL']
img_size = 150
def get_training_data(scans):
    data = list()
    for target in targets: 
        path = os.path.join(scans, target)
        class_label = targets.index(target)
        for img in os.listdir(path):
            try:
                arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(arr, (img_size, img_size))
                data.append([resized_arr, class_label])
            except Exception as e:
                print(e)
    return np.array(data)

In [105]:
#extracting the dataset
train = get_training_data('/content/gdrive/MyDrive/PythonProjectWebApp/chest_xray/train')
test = get_training_data('/content/gdrive/MyDrive/PythonProjectWebApp/chest_xray/test')
val = get_training_data('/content/gdrive/MyDrive/PythonProjectWebApp/chest_xray/val')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [106]:
#split the data into training, testing and validation groups with variables to hold the target names.
X_train = []
y_train = []

X_val = []
y_val = []

X_test = []
y_test = []

for image_scan, target in train:
    X_train.append(image_scan)
    y_train.append(target)

for image_scan, target in test:
    X_test.append(image_scan)
    y_test.append(target)
    
for image_scan, target in val:
    X_val.append(image_scan)
    y_val.append(target)

In [107]:
# Normalize data
X_train = np.array(X_train) / 255
X_val = np.array(X_val) / 255
X_test = np.array(X_test) / 255
X_test.shape

(624, 150, 150)

In [109]:

X_test = np.stack(X_test, axis=0)  
X_test = X_test.reshape(-1, img_size, img_size, 3)
y_test = np.array(y_test)
X_test.shape

(208, 150, 150, 3)

In [110]:
model.evaluate(X_test,y_test)

ValueError: ignored

In [97]:
predictions = (model.predict(X_test) > 0.56).astype("int32")
predictions = predictions.reshape(1, -1)[0]
predictions[:]

array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,